# MPNet Dual-Objective Training Pipeline## Similar Ticket Detection + Classification**Project:** ITSM Incident Management AI  **Model:** all-mpnet-base-v2 (sentence-transformers)  **Date:** December 2025---## 🎯 Training Objectives### PRIMARY: Similar Ticket DetectionTrain model to produce embeddings where similar tickets are **semantically close**.**Use Cases:**- 🔍 Find duplicate tickets- 🔗 Link related incidents for root cause analysis- 💡 Suggest similar resolved tickets to speed up resolution- 📊 Cluster tickets by problem type**Method:** Contrastive Learning- Positive pairs: Tickets from same category (similar)- Negative pairs: Tickets from different categories (dissimilar)- Loss: Contrastive loss with margin### SECONDARY: Ticket ClassificationPredict category labels for automatic routing.**Method:** Cross-Entropy Loss with classification head---## 📋 Pipeline Overview1. Data Loading & EDA2. Create Contrastive Pairs (similar/dissimilar tickets)3. Model Training with Dual Loss4. **Similarity Search Index Building**5. **Similar Ticket Retrieval Demo**6. Classification Evaluation7. Visualization & Analysis

## 1. Setup and Configuration

In [ ]:
# Core librariesimport osimport sysimport jsonimport pickleimport warningsfrom pathlib import Pathfrom datetime import datetime# Data and MLimport numpy as npimport pandas as pdimport torchimport torch.nn as nnimport torch.nn.functional as Ffrom torch.utils.data import Dataset, DataLoader# Sentence transformersfrom sentence_transformers import SentenceTransformer, losses, InputExample# Metrics and utilitiesfrom sklearn.model_selection import train_test_splitfrom sklearn.preprocessing import LabelEncoderfrom sklearn.metrics import accuracy_score, precision_recall_fscore_supportfrom sklearn.metrics.pairwise import cosine_similarity# Visualizationimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.manifold import TSNEfrom tqdm.auto import tqdm# Databaseimport psycopg2from sqlalchemy import create_enginewarnings.filterwarnings('ignore')print("✓ All libraries imported")

In [ ]:
# ConfigurationRANDOM_SEED = 42np.random.seed(RANDOM_SEED)torch.manual_seed(RANDOM_SEED)# Model settingsMODEL_NAME = 'sentence-transformers/all-mpnet-base-v2'MAX_SEQ_LENGTH = 512EMBEDDING_DIM = 768# Training hyperparametersBATCH_SIZE = 16LEARNING_RATE = 2e-5NUM_EPOCHS = 10CONTRASTIVE_MARGIN = 0.5  # For contrastive loss# DirectoriesOUTPUT_DIR = Path('/mnt/user-data/outputs')MODEL_DIR = OUTPUT_DIR / 'models'PLOTS_DIR = OUTPUT_DIR / 'plots'RESULTS_DIR = OUTPUT_DIR / 'results'for d in [MODEL_DIR, PLOTS_DIR, RESULTS_DIR]:    d.mkdir(parents=True, exist_ok=True)# Devicedevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')print(f"Using device: {device}")print(f"Model: {MODEL_NAME}")print(f"Batch size: {BATCH_SIZE}")

## 2. Data Loading

In [ ]:
def load_or_create_sample_data():    """Load from DB or create sample data"""    try:        # Try database connection        DB_HOST = os.getenv('DB_HOST', 'localhost')        DB_PORT = os.getenv('DB_PORT', '5432')        DB_NAME = os.getenv('DB_NAME', 'itsm_tickets')        DB_USER = os.getenv('DB_USER', 'user')        DB_PASSWORD = os.getenv('DB_PASSWORD', 'password')                connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'        engine = create_engine(connection_string)                query = """        SELECT ticket_id, description, category, subcategory,                service_offering, assignment_group        FROM tickets        WHERE description IS NOT NULL AND LENGTH(description) > 10        LIMIT 10000        """                df = pd.read_sql_query(query, engine)        print(f"✓ Loaded {len(df)} tickets from database")        return df            except Exception as e:        print(f"Database unavailable: {e}")        print("Creating sample data...")                # Create realistic sample data        categories = ['Access Issues', 'Hardware Issue', 'Password Reset',                      'Software Issue', 'Network Issue']                samples = []        descriptions = {            'Access Issues': [                "Cannot access email account. Login fails with authentication error.",                "User locked out of system after failed login attempts.",                "Access denied to shared folder on network drive.",            ],            'Hardware Issue': [                "Printer not responding. Paper jam error on printer HP4500.",                "Monitor display flickering. Screen shows vertical lines.",                "Keyboard keys not working properly. Multiple keys stuck.",            ],            'Password Reset': [                "Need password reset for SAP system. Locked out after failed attempts.",                "Forgot password for email account. Cannot log in.",                "Password expired for VPN access. Need immediate reset.",            ],            'Software Issue': [                "Application crashes when opening reports. Error code 0x8007000E.",                "Software freezes during data export. Have to force quit.",                "Cannot install software update. Error message appears.",            ],            'Network Issue': [                "Network connection keeps dropping. Unable to access shared drives.",                "Internet connectivity issues. Cannot browse websites.",                "VPN connection fails. Cannot connect to office network.",            ]        }                for category in categories:            for desc in descriptions[category] * 67:  # Create ~1000 samples                samples.append({                    'description': desc,                    'category': category,                    'service_offering': category.split()[0] + ' Services'                })                df = pd.DataFrame(samples)        df['ticket_id'] = [f'INC{i:06d}' for i in range(len(df))]        df['subcategory'] = df['category']        df['assignment_group'] = 'Support Team'                print(f"✓ Created {len(df)} sample tickets")        return df# Load datadf = load_or_create_sample_data()print(f"\nDataset shape: {df.shape}")print(f"Categories: {df['category'].unique()}")df.head()

## 3. Text Preprocessing

In [ ]:
import redef preprocess_text(text):    """Clean and normalize text"""    if pd.isna(text):        return ""    text = str(text)    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs    text = re.sub(r'\S+@\S+', '[EMAIL]', text)  # Mask emails    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace    return text.strip()# Apply preprocessingdf['description_clean'] = df['description'].apply(preprocess_text)df = df[df['description_clean'].str.split().str.len() >= 5].copy()# Encode labelslabel_encoder = LabelEncoder()df['category_id'] = label_encoder.fit_transform(df['category'])print(f"After preprocessing: {len(df)} tickets")print(f"Categories: {len(label_encoder.classes_)}")# Save label encoderwith open(MODEL_DIR / 'label_encoder.pkl', 'wb') as f:    pickle.dump(label_encoder, f)

## 4. Train/Val/Test Split

In [ ]:
# Stratified splittrain_val_df, test_df = train_test_split(    df, test_size=0.15, stratify=df['category_id'], random_state=RANDOM_SEED)train_df, val_df = train_test_split(    train_val_df, test_size=0.15/(1-0.15), stratify=train_val_df['category_id'],     random_state=RANDOM_SEED)print(f"Training:   {len(train_df):5d} ({len(train_df)/len(df)*100:.1f}%)")print(f"Validation: {len(val_df):5d} ({len(val_df)/len(df)*100:.1f}%)")print(f"Test:       {len(test_df):5d} ({len(test_df)/len(df)*100:.1f}%)")

## 5. Create Contrastive Pairs for Similarity Learning**This is the KEY step for learning good embeddings!**We create pairs of tickets:- **Positive pairs (label=1):** Two tickets from SAME category → should be close in embedding space- **Negative pairs (label=0):** Two tickets from DIFFERENT categories → should be far apartThe model learns to minimize distance for positive pairs and maximize distance for negative pairs.

In [ ]:
def create_contrastive_pairs(df, num_pairs_per_ticket=2):    """    Create contrastive pairs for training.        Returns:        list of (text1, text2, label) tuples        label=1 for similar, 0 for dissimilar    """    pairs = []    grouped = df.groupby('category_id')        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Creating pairs"):        text1 = row['description_clean']        category = row['category_id']                # Positive pairs: same category        same_cat = grouped.get_group(category)        same_cat = same_cat[same_cat.index != idx]                if len(same_cat) > 0:            n_pos = min(num_pairs_per_ticket, len(same_cat))            pos_samples = same_cat.sample(n=n_pos, random_state=RANDOM_SEED)            for _, pos_row in pos_samples.iterrows():                pairs.append((text1, pos_row['description_clean'], 1))                # Negative pairs: different category          diff_cat = df[df['category_id'] != category]        if len(diff_cat) > 0:            n_neg = min(num_pairs_per_ticket, len(diff_cat))            neg_samples = diff_cat.sample(n=n_neg, random_state=RANDOM_SEED)            for _, neg_row in neg_samples.iterrows():                pairs.append((text1, neg_row['description_clean'], 0))        return pairs# Create pairsprint("Creating contrastive pairs...")train_pairs = create_contrastive_pairs(train_df, num_pairs_per_ticket=2)val_pairs = create_contrastive_pairs(val_df, num_pairs_per_ticket=1)print(f"\nContrastive Pairs Created:")print(f"  Training: {len(train_pairs)} pairs")print(f"  - Positive (similar): {sum(1 for p in train_pairs if p[2]==1)}")print(f"  - Negative (dissimilar): {sum(1 for p in train_pairs if p[2]==0)}")print(f"  Validation: {len(val_pairs)} pairs")

## 6. Model Training with Contrastive LossWe use Sentence-Transformers library which handles:- MPNet encoding- Contrastive loss computation  - Efficient batch processing

In [ ]:
from sentence_transformers import InputExample, lossesfrom torch.utils.data import DataLoader# Load pre-trained modelmodel = SentenceTransformer(MODEL_NAME, device=device)print(f"✓ Loaded model: {MODEL_NAME}")# Convert pairs to InputExample formattrain_examples = [    InputExample(texts=[text1, text2], label=float(label))    for text1, text2, label in train_pairs]val_examples = [    InputExample(texts=[text1, text2], label=float(label))    for text1, text2, label in val_pairs]# Create dataloaderstrain_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)# Use OnlineContrastiveLoss - perfect for our use case!train_loss = losses.OnlineContrastiveLoss(    model=model,    margin=CONTRASTIVE_MARGIN)print(f"\nTraining Setup:")print(f"  - Loss: OnlineContrastiveLoss (margin={CONTRASTIVE_MARGIN})")print(f"  - Batch size: {BATCH_SIZE}")print(f"  - Batches per epoch: {len(train_dataloader)}")print(f"  - Total training steps: {len(train_dataloader) * NUM_EPOCHS}")

In [ ]:
# Train modelprint("\n" + "="*60)print("Starting Training")print("="*60)model.fit(    train_objectives=[(train_dataloader, train_loss)],    epochs=NUM_EPOCHS,    warmup_steps=100,    output_path=str(MODEL_DIR / 'mpnet_similarity_model'),    show_progress_bar=True,    save_best_model=True)print("\n✓ Training complete!")print(f"Model saved to: {MODEL_DIR / 'mpnet_similarity_model'}

## 7. Build Similarity Search IndexNow we encode ALL training tickets to create a searchable index.

In [ ]:
print("Building similarity search index...")print("="*60)# Get all training ticketsall_train_texts = train_df['description_clean'].tolist()all_train_ids = train_df['ticket_id'].tolist()all_train_categories = train_df['category'].tolist()print(f"Encoding {len(all_train_texts)} tickets...")train_embeddings = model.encode(    all_train_texts,    show_progress_bar=True,    convert_to_tensor=False,    normalize_embeddings=True  # Normalize for cosine similarity)# Create searchable indexembedding_index = {    'embeddings': train_embeddings,    'ticket_ids': all_train_ids,    'texts': all_train_texts,    'categories': all_train_categories}# Save indexwith open(MODEL_DIR / 'embedding_index.pkl', 'wb') as f:    pickle.dump(embedding_index, f)print(f"\n✓ Index created!")print(f"  - Size: {len(all_train_texts)} tickets")print(f"  - Dimensions: {train_embeddings.shape}")print(f"  - Saved to: {MODEL_DIR / 'embedding_index.pkl'}

## 8. 🔍 Similar Ticket Search Demo**This is what we trained for!** Let's find similar tickets.

In [ ]:
def find_similar_tickets(query_text, embedding_index, model, top_k=5):    """    Find most similar tickets to query.        Returns:        list of dicts with ticket info and similarity scores    """    # Encode query    query_emb = model.encode(        [query_text],        convert_to_tensor=False,        normalize_embeddings=True    )        # Compute cosine similarity    similarities = cosine_similarity(query_emb, embedding_index['embeddings'])[0]        # Get top-k    top_indices = np.argsort(similarities)[::-1][:top_k]        results = []    for idx in top_indices:        results.append({            'ticket_id': embedding_index['ticket_ids'][idx],            'description': embedding_index['texts'][idx],            'category': embedding_index['categories'][idx],            'similarity': float(similarities[idx])        })        return resultsprint("SIMILAR TICKET SEARCH DEMO")print("="*60)# Try examples from test setfor i in range(3):    query = test_df.iloc[i]    print(f"\n{'─'*60}")    print(f"QUERY TICKET #{i+1}: {query['ticket_id']}")    print(f"Category: {query['category']}")    print(f"Description: {query['description_clean'][:100]}...")    print(f"\nTop 5 Similar Tickets:")    print(f"{'─'*60}")        similar = find_similar_tickets(        query['description_clean'],         embedding_index,         model,         top_k=5    )        for rank, ticket in enumerate(similar, 1):        match = "✓" if ticket['category'] == query['category'] else "✗"        print(f"\n{rank}. {ticket['ticket_id']} | Similarity: {ticket['similarity']:.4f} {match}")        print(f"   Category: {ticket['category']}")        print(f"   Description: {ticket['description'][:80]}...")print(f"\n{'='*60}")

## 9. Evaluate Similarity Retrieval QualityMeasure how well the model finds similar tickets using:- **Mean Reciprocal Rank (MRR):** Average rank of first relevant result- **Precision@K:** % of relevant results in top-K

In [ ]:
def evaluate_similarity_retrieval(test_df, embedding_index, model, k=10):    """Evaluate with MRR and Precision@K"""    reciprocal_ranks = []    precisions = []        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):        query_text = row['description_clean']        query_category = row['category']                # Find similar        similar = find_similar_tickets(query_text, embedding_index, model, top_k=k)                # Calculate MRR        first_correct_rank = None        for rank, ticket in enumerate(similar, 1):            if ticket['category'] == query_category:                first_correct_rank = rank                break                if first_correct_rank:            reciprocal_ranks.append(1.0 / first_correct_rank)        else:            reciprocal_ranks.append(0.0)                # Calculate Precision@K        correct = sum(1 for t in similar if t['category'] == query_category)        precisions.append(correct / k)        return np.mean(reciprocal_ranks), np.mean(precisions)# Evaluateprint("Evaluating similarity retrieval...")mrr, precision_10 = evaluate_similarity_retrieval(test_df, embedding_index, model, k=10)print(f"\n{'='*60}")print("SIMILARITY RETRIEVAL RESULTS")print(f"{'='*60}")print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")print(f"Precision@10:               {precision_10:.4f}")print(f"\nInterpretation:")print(f"  - First relevant result appears at rank ~{1/mrr:.1f} on average")print(f"  - {precision_10*100:.1f}% of top-10 results are relevant")if mrr > 0.75:    print(f"\n✓ EXCELLENT: MRR > 0.75 (Target achieved!)")elif mrr > 0.60:    print(f"\n✓ GOOD: MRR > 0.60 (Close to target)")else:    print(f"\n⚠ Needs improvement (Target: MRR > 0.75)")print(f"{'='*60}")

## 10. Visualization: t-SNE Embedding Space

In [ ]:
# Sample for visualizationsample_size = min(500, len(test_df))sample_texts = test_df['description_clean'].sample(sample_size, random_state=RANDOM_SEED)sample_labels = test_df.loc[sample_texts.index, 'category']print(f"Computing embeddings for {sample_size} samples...")embeddings = model.encode(sample_texts.tolist(), show_progress_bar=True)print("Computing t-SNE projection...")tsne = TSNE(n_components=2, random_state=RANDOM_SEED, perplexity=30)embeddings_2d = tsne.fit_transform(embeddings)# Plotfig, ax = plt.subplots(figsize=(12, 10))unique_cats = sample_labels.unique()colors = plt.cm.tab10(np.linspace(0, 1, len(unique_cats)))for cat, color in zip(unique_cats, colors):    mask = sample_labels == cat    ax.scatter(        embeddings_2d[mask, 0],        embeddings_2d[mask, 1],        c=[color],        label=cat,        alpha=0.6,        s=50    )ax.set_xlabel('t-SNE Dimension 1', fontsize=12, fontweight='bold')ax.set_ylabel('t-SNE Dimension 2', fontsize=12, fontweight='bold')ax.set_title('t-SNE: Ticket Embeddings by Category', fontsize=14, fontweight='bold')ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')ax.grid(True, alpha=0.3)plt.tight_layout()plt.savefig(PLOTS_DIR / 'tsne_embeddings.png', dpi=300, bbox_inches='tight')plt.show()print(f"✓ Saved to {PLOTS_DIR / 'tsne_embeddings.png'}

## 11. Summary & Next Steps### ✅ Completed:- [x] Loaded and preprocessed ticket data- [x] Created contrastive pairs for similarity learning- [x] Trained MPNet with contrastive loss- [x] Built searchable embedding index- [x] Demonstrated similar ticket search- [x] Evaluated retrieval quality (MRR, Precision@K)- [x] Visualized embedding space### 🚀 Next Steps:1. **Deploy similarity search API** - Expose find_similar_tickets() as REST endpoint2. **Integrate with ServiceNow** - Auto-link similar tickets when created3. **Add classification head** - For category prediction (secondary objective)4. **Implement caching** - Store embeddings for fast retrieval5. **Monitor performance** - Track MRR in production6. **Fine-tune on feedback** - Retrain with user relevance signals### 💾 Saved Artifacts:- Model: `models/mpnet_similarity_model/`- Embedding index: `models/embedding_index.pkl`- Label encoder: `models/label_encoder.pkl`- Visualizations: `plots/`

In [ ]:
# Save final metricsmetrics = {    'mrr': float(mrr),    'precision_at_10': float(precision_10),    'num_train_tickets': len(train_df),    'num_test_tickets': len(test_df),    'num_categories': len(label_encoder.classes_),    'model_name': MODEL_NAME,    'training_date': datetime.now().isoformat()}with open(RESULTS_DIR / 'similarity_metrics.json', 'w') as f:    json.dump(metrics, f, indent=2)print("="*60)print("TRAINING COMPLETE!")print("="*60)print(f"Model saved to: {MODEL_DIR}")print(f"Results saved to: {RESULTS_DIR}")print(f"\nKey Metrics:")print(f"  - MRR: {mrr:.4f}")print(f"  - Precision@10: {precision_10:.4f}")print("="*60)